<a href="https://colab.research.google.com/github/unionxic/Mogakso/blob/main/yolo11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글 드라이브 마운트 (데이터 저장용)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install ultralytics kaggle

In [ ]:
import torch
print("GPU 사용 가능 여부:", torch.cuda.is_available())


In [ ]:
from ultralytics import YOLO

# YOLO 분류 모델 로드 (가정: "yolo11-cls.pt" 파일이 존재해야 함)
model = YOLO("yolo11n-cls.pt")

# 모델 학습
results = model.train(
    data="/content/drive/MyDrive/data-v1/",  # 올바른 절대 경로 사용
    epochs=30,                 # 원하는 에포크 수 조정
    imgsz=224,                 # 분류 모델에 적합한 이미지 크기 (예: 224, 256)
    project="/content/runs",    # 결과 저장 폴더
    name="yolo11_cls_experiment"
)


In [ ]:
from ultralytics import YOLO

# 모델 로드
model = YOLO("/content/runs/yolo11_cls_experiment4/weights/best.pt")

# 개별 하위 폴더별로 예측 실행
folders = [
    "/content/drive/MyDrive/data-v1/test/moderate-accident",
    "/content/drive/MyDrive/data-v1/test/no-accident",
    "/content/drive/MyDrive/data-v1/test/severe-accident"
]

for folder in folders:
    results = model.predict(source=folder, save=True)
    print(f"Predictions for {folder}:")
    print(results)


In [ ]:
import matplotlib.pyplot as plt
import cv2
import os

# 예측된 결과를 시각화하는 함수
def visualize_predictions(results):
    for result in results[:5]:  # 처음 5개만 예제 출력
        predicted_class = result.names[result.probs.top1]  # 예측된 클래스
        confidence = result.probs.data[result.probs.top1].item()  # 확률

        # 원본 이미지 불러오기
        image = cv2.imread(result.path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 이미지 표시
        plt.figure(figsize=(6,6))
        plt.imshow(image)
        plt.axis("off")
        plt.title(f"Predicted: {predicted_class} ({confidence:.2%})", fontsize=14)
        plt.show()

# 모든 폴더에 대해 결과 시각화
for folder in folders:
    print(f"\n📌 예측된 결과 - {folder}")
    visualize_predictions(model.predict(source=folder))
